**In order to get the results, you need to run everything up to Model Training and Results Section, and then run all the code blocks with the face2sketch comment for the face2sketch training and results, and after getting the results, execute the sketch2face blocks for the sketch2face results.**

We start by importing the used libraries

In [1]:
#From the week 3 lab files, but a little tweked so in has more depth
import numpy as np
import tensorflow as tf
import keras 
from keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Dropout, Input
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
import os
import re

In [2]:
#https://drive.google.com/drive/folders/1C2lQ61MssBmVjRxrzYy0vqDOMWWxSEex?usp=sharing Get the files into your drive and open from there
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)


# defining the size of image 
SIZE = 128

image_path = "/content/gdrive/MyDrive/Data_for_CW1/photos"
img_array = []

sketch_path = '/content/gdrive/MyDrive/Data_for_CW1/sketches'
sketch_array = []


image_file = sorted_alphanumeric(os.listdir(image_path))
sketch_file = sorted_alphanumeric(os.listdir(sketch_path))

Load data.

We convert the images to arrays ansd then we store them in a list

Data augumentation is also performed due to the small amount of provided samples

In [4]:
for i in tqdm(image_file):
    image = cv2.imread(image_path + '/' + i,1)
    
    # as opencv load image in bgr format converting it to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing images 
    image = cv2.resize(image, (SIZE, SIZE))
    
    # normalizing image 
    image = image.astype('float32') / 255.0
    
    #appending normal normal image    
    img_array.append(img_to_array(image))
    # Image Augmentation
    
    # horizontal flip 
    img1 = cv2.flip(image,1)
    img_array.append(img_to_array(img1))
     #vertical flip 
    img2 = cv2.flip(image,-1)
    img_array.append(img_to_array(img2))
     #vertical flip 
    img3 = cv2.flip(image,-1)
    # horizontal flip
    img3 = cv2.flip(img3,1)
    img_array.append(img_to_array(img3))
    # rotate clockwise 
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    img_array.append(img_to_array(img4))
    # flip rotated image 
    img5 = cv2.flip(img4,1)
    img_array.append(img_to_array(img5))
     # rotate anti clockwise 
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    img_array.append(img_to_array(img6))
    # flip rotated image 
    img7 = cv2.flip(img6,1)
    img_array.append(img_to_array(img7))
  
    
for i in tqdm(sketch_file):
    image = cv2.imread(sketch_path + '/' + i,1)
    
    # as opencv load image in bgr format converting it to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing images 
    image = cv2.resize(image, (SIZE, SIZE))
    
    # normalizing image 
    image = image.astype('float32') / 255.0
    # appending normal sketch image
    sketch_array.append(img_to_array(image))
    
    #Image Augmentation
    # horizontal flip 
    img1 = cv2.flip(image,1)
    sketch_array.append(img_to_array(img1))
     #vertical flip 
    img2 = cv2.flip(image,-1)
    sketch_array.append(img_to_array(img2))
     #vertical flip 
    img3 = cv2.flip(image,-1)
    # horizontal flip
    img3 = cv2.flip(img3,1)
    sketch_array.append(img_to_array(img3))
    # rotate clockwise 
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    sketch_array.append(img_to_array(img4))
    # flip rotated image 
    img5 = cv2.flip(img4,1)
    sketch_array.append(img_to_array(img5))
     # rotate anti clockwise 
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    sketch_array.append(img_to_array(img6))
    # flip rotated image 
    img7 = cv2.flip(img6,1)
    sketch_array.append(img_to_array(img7))

100%|██████████| 188/188 [01:53<00:00,  1.65it/s]


In [5]:
print("Total number of sketch images:",len(sketch_array))
print("Total number of images:",len(img_array))
print(len(img_array[1]))

Total number of sketch images: 1504
Total number of images: 1504
128


Slicing and reshaping

Out of 1504 images I have sliced them to two part. train images consist 1400 images while test images contains 104 images. After slicing image array, I reshaped them so that images can be fed directly into our encoder network

In [6]:
train_sketch_image = sketch_array[:1400]
train_image = img_array[:1400]
test_sketch_image = sketch_array[1400:]
test_image = img_array[1400:]

train_sketch_image = np.array(train_sketch_image)
train_image = np.array(train_image)
test_sketch_image = np.array(test_sketch_image)
test_image = np.array(test_image)


print(train_sketch_image.shape)
print(train_image.shape)
print(test_sketch_image.shape)
print(test_image.shape)

(1400, 128, 128, 3)
(1400, 128, 128, 3)
(104, 128, 128, 3)
(104, 128, 128, 3)


In [7]:
# reshaping
train_sketch_image = train_sketch_image.reshape((len(train_sketch_image), np.prod(train_sketch_image.shape[1:])))
train_image = train_image.reshape((len(train_image), np.prod(train_image.shape[1:])))
print('Train color image shape:',train_image.shape)
test_sketch_image = test_sketch_image.reshape((len(test_sketch_image), np.prod(test_sketch_image.shape[1:])))
test_image = test_image.reshape((len(test_image), np.prod(test_image.shape[1:])))
print('Test color image shape',test_image.shape)

Train color image shape: (1400, 49152)
Test color image shape (104, 49152)


In [ ]:
np.shape(test_sketch_image)

(104, 49152)

In [8]:
#Load the model if you dont want to run the process again
from tensorflow.keras.models import load_model
model = load_model('/content/gdrive/MyDrive/Data_for_CW1/MyModels/DAE/sketch2faceDAE.h5')

**Model**

In [ ]:
# define a 8x8 deep autoencoder
input_img = keras.Input(shape=(49152,))
encoded = keras.layers.Dense(4096, activation='relu')(input_img)
encoded = keras.layers.Dense(2048, activation='relu')(encoded)
encoded = keras.layers.Dense(1024, activation='relu')(encoded)
encoded = keras.layers.Dense(512, activation='relu')(encoded)
encoded = keras.layers.Dense(256, activation='relu')(encoded)
encoded = keras.layers.Dense(128, activation='relu')(encoded)
encoded = keras.layers.Dense(64, activation='relu')(encoded)
encoded = keras.layers.Dense(32, activation='relu')(encoded)

decoded = keras.layers.Dense(64, activation='relu')(encoded)
decoded = keras.layers.Dense(128, activation='relu')(decoded)
decoded = keras.layers.Dense(256, activation='relu')(decoded)
decoded = keras.layers.Dense(512, activation='relu')(decoded)
decoded = keras.layers.Dense(1024, activation='relu')(decoded)
decoded = keras.layers.Dense(2048, activation='relu')(decoded)
decoded = keras.layers.Dense(4096, activation='relu')(decoded)
decoded = keras.layers.Dense(49152, activation='sigmoid')(decoded)

model = keras.Model(input_img, decoded)


To get summary of model


In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 49152)]           0         
                                                                 
 dense (Dense)               (None, 4096)              201330688 
                                                                 
 dense_1 (Dense)             (None, 2048)              8390656   
                                                                 
 dense_2 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (None, 512)               524800    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 128)               32896 

# Model Training and Results

Compiling and Fitting our model

Here i have used Adam optimizer and mean_absolute_error as loss and have trained model for 100 epochs

In [ ]:
#Face2sketch
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['acc'])

hist = model.fit(train_image, train_sketch_image,epochs=100, verbose = 1)

plt.plot(hist.history["loss"]);
plt.xlabel('Epochs');
plt.ylabel('Training Error');

Evaluating our face2sketch model

In [10]:
#face2sketch
prediction_on_test_data = model.evaluate(test_image, test_sketch_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[0] * 100,1))

4/4 [==============================] - 4s 31ms/step - loss: 0.0325 - acc: 0.0000e+00
Loss:  0.03245147317647934
Accuracy:  3.2


In [ ]:
#sketch2face
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['acc'])

hist = model.fit(train_sketch_image, train_image, epochs=100,verbose = 1)

plt.plot(hist.history["loss"]);
plt.xlabel('Epochs');
plt.ylabel('Training Error');

Evaluating our sketch2face model

In [9]:
#sketch2face
#If you run it after the face2sketch training the dimentions change for the evaluation so we change it back
train_sketch_image = train_sketch_image.reshape((len(train_sketch_image), np.prod(train_sketch_image.shape[1:])))
train_image = train_image.reshape((len(train_image), np.prod(train_image.shape[1:])))
print('Train color image shape:',train_image.shape)
test_sketch_image = test_sketch_image.reshape((len(test_sketch_image), np.prod(test_sketch_image.shape[1:])))
test_image = test_image.reshape((len(test_image), np.prod(test_image.shape[1:])))
print('Test color image shape',test_image.shape)

prediction_on_test_data = model.evaluate(test_sketch_image, test_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

Train color image shape: (1400, 49152)
Test color image shape (104, 49152)
4/4 [==============================] - 2s 22ms/step - loss: 0.0196 - acc: 0.0000e+00
Loss:  0.01958073116838932
Accuracy:  0.0


Plotting our predicted sketch along with real sketch

In [11]:
#face2sketch
encoded_imgs = model.predict(test_image)

In [10]:
#sketch2face
encoded_imgs2 = model.predict(test_sketch_image)

In [19]:
#face2sketch
def show_images(real,sketch, predicted):
    plt.figure(figsize = (12,12))
    plt.subplot(1,3,1)
    plt.title("Image",fontsize = 15, color = 'Lime')
    plt.imshow(real)
    plt.subplot(1,3,2)
    plt.title("Sketch",fontsize = 15, color = 'Blue')
    plt.imshow(sketch)
    plt.subplot(1,3,3)
    plt.title("Predicted",fontsize = 15, color = 'gold')
    plt.imshow(predicted)

In [11]:
#sketch2face
def show_images2(real,image, predicted):
    plt.figure(figsize = (12,12))
    plt.subplot(1,3,1)
    plt.title("Sketch",fontsize = 15, color = 'Lime')
    plt.imshow(real)
    plt.subplot(1,3,2)
    plt.title("Image",fontsize = 15, color = 'Blue')
    plt.imshow(image)
    plt.subplot(1,3,3)
    plt.title("Predicted",fontsize = 15, color = 'gold')
    plt.imshow(predicted)

In [12]:
#face2sketch and sketch2face (run for both models)
test_sketch_image = test_sketch_image.reshape(104, 128, 128, 3)
test_image = test_image.reshape(104, 128, 128, 3)
np.shape(test_sketch_image[1])


(128, 128, 3)

In [46]:
#face2sketch
ls = [i for i in range(0,95,8)]
pred1 = []
for i in ls:
    predicted =np.clip(encoded_imgs[i].reshape(SIZE,SIZE,3),0.0,1.0)
    pred1.append(predicted)
    test_sketch_image[i] = test_sketch_image[i].reshape(128, 128, 3)
    test_image[i] = test_image[i].reshape(128, 128, 3)
    show_images(test_image[i],test_sketch_image[i],predicted)

Output hidden; open in https://colab.research.google.com to view.

In [14]:
#!pip install image-similarity-measures
#encoded_imgs1 or 2 depending on face2sketch or the opposite
pred1 = []
for i in range(0,104):
    predicted =np.clip(encoded_imgs2[i].reshape(SIZE,SIZE,3),0.0,1.0)
    pred1.append(predicted)

In [15]:
from skimage.metrics import structural_similarity as ssim1
SSIMarray = []
sumssim = 0
for i in range(0, 104):
  SSIM = ssim1(test_image[i], pred1[i], multichannel=True)
  SSIMarray.append(SSIM)
  sumssim+= SSIM

avgssim = sumssim/104
print("SSIM =",avgssim)

from sklearn.metrics import mean_squared_error as mse1
import math
RMSEarray = []
sumrmse = 0

t = np.reshape(test_image, (104,-1))
p = np.reshape(pred1, (104,-1))

for i in range(0, 104):
  MSE = mse1(t[i], p[i])
  RMSEarray.append(math.sqrt(MSE))
  sumrmse+= math.sqrt(MSE)

avgrmse = sumrmse/104
print("RMSE =",avgrmse)

from sklearn.metrics import median_absolute_error as mae

maearray = []
summae = 0

for i in range(0, 104):
  MAE = mae(t[i], p[i])
  maearray.append(MAE)
  summae+= MAE

avgrmae = summae/104
print("MAE =",avgrmae)

SSIM = 0.7350823430964094
RMSE = 0.1361550913388706
MAE = 0.06467520169770488


In [16]:
#sketch2face
ls = [i for i in range(0,95,8)]
pred2 = []
for i in ls:
    predicted =np.clip(encoded_imgs2[i].reshape(SIZE,SIZE,3),0.0,1.0)
    pred2.append(predicted)
    test_sketch_image[i] = test_sketch_image[i].reshape(128, 128, 3)
    test_image[i] = test_image[i].reshape(128, 128, 3)
    show_images2(test_sketch_image[i],test_image[i],predicted)

Output hidden; open in https://colab.research.google.com to view.

Save Models for reproducibility

In [ ]:
#face2sketch
#model.save('/content/gdrive/MyDrive/MyModels/DAE/face2sketchDAE.h5')

In [ ]:
#sketch2face
#model.save("/content/gdrive/MyDrive/MyModels/DAE/sketch2faceDAE.h5")